In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [57]:
class PDBfile():
    global aa321dict
    aa321dict = {
                   "CYS" : 'C',
                   "ASP" : 'D',
                   "SER" : 'S',
                   "ASN" : 'N',
                   "LYS" : 'K',
                   "ILE" : 'I',
                   "PRO" : 'P',
                   "THR" : 'T',
                   "PHE" : 'F',
                   "GLN" : 'Q',
                   "GLY" : 'G',
                   "HIS" : 'H',
                   "LEU" : 'L',
                   "ARG" : 'R',
                   "TRP" : 'W',
                   "ALA" : 'A',
                   "VAL" : 'V',
                   "GLU" : 'E',
                   "TYR" : 'Y',
                   "MET" : 'M'
                }
    
    global pdb_format
    pdb_format = '6s5s1s4s1s3s1s1s4s1s3s8s8s8s6s6s6s4s2s3s'
    import numpy as np
    global np
    
    
    from sklearn import neighbors
    global neighbors
    
    def SplitChain():
        return 0
    
    def GetCaCoordDict(pdbfilename):
        from struct import unpack
        
        CaCoordDict={}
        pdbfile = open(pdbfilename)
        for line in pdbfile:
            if line[0:4] == "ATOM":
                col = unpack(pdb_format, line.encode())
                remark = str(col[0].decode()).strip()
                atomnum = int(col[1].decode())
                atomtype = str(col[3].decode()).strip()
                resname = str(col[5].decode()).strip()
                chainid = str(col[7].decode()).strip()
                seqnum = int(col[8].decode())
                coord_x = float(col[11].decode())
                coord_y = float(col[12].decode())
                coord_z = float(col[13].decode())
                if atomtype == "CA":
                    CaName = chainid+"_"+str(seqnum)+"_"+resname+"_CA"
                    CaCoordDict[(coord_x,coord_y,coord_z)] = CaName
                
                
                #print(col)
        return CaCoordDict
    
    def Get15nearestAnimoAcids(centreAACACoord,CaCoordDict):
        AAlist = []
        X = np.array(list(CaCoordDict.keys()))
        tree = neighbors.KDTree(X, leaf_size=2)              # doctest: +SKIP
        p = np.array([centreAACACoord])
        dist, ind = tree.query(p, k=16)                # doctest: +SKIP
        for x in X[ind[0]]:
            #print(x)
            AAlist.append(aa321dict[CaCoordDict[tuple(x)].split("_")[2]])
        return AAlist
    
    def GetResCaCoordDict(pdbfilename,chain):
        from struct import unpack
        
        ResCaCoordDict={}
        pdbfile = open(pdbfilename)
        #mutseqnum = int(mutation[1:-1])
        #wildtype = mutation[0]
        for line in pdbfile:
            if line[0:4] == "ATOM":
                col = unpack(pdb_format, line.encode())
                remark = str(col[0].decode()).strip()
                atomnum = int(col[1].decode())
                atomtype = str(col[3].decode()).strip()
                resname = str(col[5].decode()).strip()
                chainid = str(col[7].decode()).strip()
                if chainid == chain:
                    seqnum = int(col[8].decode())
                    coord_x = float(col[11].decode())
                    coord_y = float(col[12].decode())
                    coord_z = float(col[13].decode())
                    if atomtype == "CA":
                        ResCaCoordDict[(seqnum,aa321dict[resname])] = (coord_x,coord_y,coord_z)
                        #if seqnum == mutseqnum:
                            #if wildtype != aa321dict[resname]:
                                #print("Warning!!! There might be missmatch between the seqnum and resnum! Check your pdb and mutation data carefully!")
                        #CaName = chainid+"_"+str(seqnum)+"_"+resname+"_CA"
                            #else:
        return ResCaCoordDict
    
    def GetCACoord(mutation,ResCaCoordDict):
        mutseqnum = int(mutation[1:-1])
        wildtypeAA = mutation[0]
        #mutantAA = mutation[-1]
        #print((mutseqnum,wildtypeAA))
        mutationCACoord = ResCaCoordDict.get((mutseqnum,wildtypeAA))
        if mutationCACoord == None:
            print("Warning!!! There might be missmatch between the seqnum and resnum! Check your pdb and mutation data carefully!")
        else:
            return mutationCACoord
        
class GetFeature():
    import pandas as pd
    import numpy as np
    #def ReadAAParam():
    global AA_data_df
    AA_data_df = pd.read_csv("AA_data.csv",sep=",",index_col="residue")
        #return AA_data_df
    
    def GetMutationFeature_num(AAlist):
        featurelist_num = []
        for aa in AAlist:
            featurelist_num += list(AA_data_df.loc[aa])
        return featurelist_num
    
    def GetMutationFeature_sub(mutation):
        
        featurelist_sub = []
        AA_list = ["A","C","D","E","F","G","H","I","K","L","M","N","P","Q","R","S","T","V","W","Y"]
        AA_class_list = [["G", "A", "P", "V", "L", "I", "M"],["F", "W", "Y"],["S","T", "C", "N", "Q"],["K", "R", "H"],["D", "E"]]
        wildtype = mutation[0]
        muttype = mutation[-1]
        mutationlist = []
        for aa in AA_list:
            
            i = 0
            if aa == wildtype:
                i = i - 1
            if aa == muttype:
                i = i + 1
            #print(i,aa)
            featurelist_sub.append(i)
            #print(i,aa)
        for classes in AA_class_list:
            i = 0
            if wildtype in classes:
                i = i - 1
            if muttype in classes:
                i = i + 1
            featurelist_sub.append(i)
                
            
        return featurelist_sub
        
    
                                
                      
dat_5xh3 = open("5xh3.dat")
pdbfilename = "5xh3.pdb"
ChainID = "A"
Featurelist = []
dTm_list = []
dTmb_list = []
for line in dat_5xh3:
    mutation = line.split("\t")[0]
    mutation = mutation[0]+str(int(mutation[1:-1])-29)+mutation[-1]
    dTm = float(line.split("\t")[1])
    dTm_list.append(dTm)
    if dTm < 1:
        dTmb = 0
    else:
        dTmb = 1
    dTmb_list.append(dTmb)
    CaCoordDict = PDBfile.GetCaCoordDict(pdbfilename)
    ResCaCoordDict = PDBfile.GetResCaCoordDict(pdbfilename,ChainID)
    #print(ResCaCoordDict)
    centreAACACoord = PDBfile.GetCACoord(mutation,ResCaCoordDict)
    AAlist = PDBfile.Get15nearestAnimoAcids(centreAACACoord,CaCoordDict)
    AAlist.append(mutation[-1])
    Featurelist.append(GetFeature.GetMutationFeature_num(AAlist)+GetFeature.GetMutationFeature_sub(mutation))
    
X = np.array(Featurelist)
y = np.array(dTmb_list)
print(y)

[0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 1 1 0 1 0 0 0 0 1 0 0
 1 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 1 0 1 0 0 0 0 1 1 0 0 0 0 0 1 0 1 1]


In [49]:
dat_5xh3 = open("1nww.dat")
pdbfilename = "1nww.pdb"
ChainID = "A"
Featurelist = []
dTm_list = []
dTmb_list = []
for line in dat_5xh3:
    mutation = line.split("\t")[0]
    #mutation = mutation[0]+str(int(mutation[1:-1])-29)+mutation[-1]
    dTm = float(line.split("\t")[1])
    dTm_list.append(dTm)
    if dTm < 1:
        dTmb = 0
    else:
        dTmb = 1
    dTmb_list.append(dTmb)
    CaCoordDict = PDBfile.GetCaCoordDict(pdbfilename)
    ResCaCoordDict = PDBfile.GetResCaCoordDict(pdbfilename,ChainID)
    #print(ResCaCoordDict)
    centreAACACoord = PDBfile.GetCACoord(mutation,ResCaCoordDict)
    AAlist = PDBfile.Get15nearestAnimoAcids(centreAACACoord,CaCoordDict)
    AAlist.append(mutation[-1])
    Featurelist.append(GetFeature.GetMutationFeature_num(AAlist)+GetFeature.GetMutationFeature_sub(mutation))
    
X = np.array(Featurelist)
y = np.array(dTmb_list)
print(y)

[0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0 1 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0]


In [11]:
from sklearn.svm import SVC
import matplotlib.pyplot as plt
#import plotly.express as px
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score , precision_score , roc_auc_score ,roc_curve
from sklearn.metrics import confusion_matrix

In [58]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
# One hot encoding
enc = OneHotEncoder()
Y = y

# Scale data to have mean 0 and variance 1 
# which is importance for convergence of the neural network
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data set into training and testing
X_train, X_test, Y_train, Y_test = train_test_split(
    X_scaled, Y, test_size=0.4, random_state=2)

In [47]:
import keras
import tensorflow as tf
from keras import optimizers
from keras import models
from keras import layers
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.layers import Dense
#history_dict = {}

# TensorBoard Callback
cb = TensorBoard()

model = Sequential()
model.add(Dense(72, input_dim=144, activation='sigmoid'))

model.add(Dense(36, activation='sigmoid'))
#model.add(Dense(20, activation='relu'))
model.add(Dense(18, activation='sigmoid'))
#model.add(Dense(5, activation='relu'))
model.add(Dense(9, activation='sigmoid'))
#model.add(Dense(3, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=300, batch_size=50,verbose=False)


In [59]:
r = model.predict(X_test)
pred = []
for x in r:
    if x <= 0.5:
        pred.append(0)
    else:
        pred.append(1)

print(classification_report(Y_test,pred))

              precision    recall  f1-score   support

           0       0.83      0.75      0.79        20
           1       0.55      0.67      0.60         9

    accuracy                           0.72        29
   macro avg       0.69      0.71      0.69        29
weighted avg       0.74      0.72      0.73        29



In [56]:
r = model.predict(X_scaled)
pred = []
for x in r:
    if x <= 0.5:
        pred.append(0)
    else:
        pred.append(1)

print("5xh3 model on 1nww data:\n"+classification_report(y,pred))

5xh3 model on 1nww data:
              precision    recall  f1-score   support

           0       0.73      0.60      0.66        50
           1       0.17      0.27      0.21        15

    accuracy                           0.52        65
   macro avg       0.45      0.43      0.43        65
weighted avg       0.60      0.52      0.55        65

